<a href="https://colab.research.google.com/github/lixinso/sample-intro-to-pytorch/blob/master/multiple_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html
# https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html